<a href="https://colab.research.google.com/github/nooralight/Conversational_Chatbot/blob/main/Chatbot_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re


lines = open('movie_lines.txt', encoding='utf-8',
             errors='ignore').read().split('\n')

convers = open('movie_conversations.txt', encoding='utf-8',
             errors='ignore').read().split('\n')


exchn = []

for conver in range(len(convers)):
    exchn.append(convers[conver].split(' +++$+++ ')[-1][1:-1].replace("'", " ").replace(",","").split())

diag = {}
for line in range(len(lines)):
    diag[lines[line].split(' +++$+++ ')[0]] = lines[line].split(' +++$+++ ')[-1]



questions = []
answers = []

for conver in exchn:
    for i in range(len(conver) - 1):
        questions.append(diag[conver[i]])
        answers.append(diag[conver[i+1]])



sorted_ques = []
sorted_ans = []
q_len= len(questions)
for i in range(q_len):
    if len(questions[i]) < 15:
        sorted_ques.append(questions[i])
        sorted_ans.append(answers[i])



def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r"i'm", "i am", txt)
    txt = re.sub(r"he's", "he is", txt)
    txt = re.sub(r"she's", "she is", txt)
    txt = re.sub(r"that's", "that is", txt)
    txt = re.sub(r"what's", "what is", txt)
    txt = re.sub(r"where's", "where is", txt)
    txt = re.sub(r"\'ll", " will", txt)
    txt = re.sub(r"\'ve", " have", txt)
    txt = re.sub(r"\'re", " are", txt)
    txt = re.sub(r"\'d", " would", txt)
    txt = re.sub(r"won't", "will not", txt)
    txt = re.sub(r"can't", "can not", txt)
    txt = re.sub(r"[^\w\s]", "", txt)
    return txt

clean_ques = []
clean_ans = []

for line in range(len(sorted_ques)):
    clean_ques.append(clean_text(sorted_ques[line]))
        
for line in range(len(sorted_ans)):
    clean_ans.append(clean_text(sorted_ans[line]))





for i in range(len(clean_ans)):
    clean_ans[i] = ' '.join(clean_ans[i].split()[:15])




clean_ans=clean_ans[:21000]
clean_ques=clean_ques[:21000]


word2count = {}

for line in range(len(clean_ques)):
    for word in clean_ques[line].split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
for line in range(len(clean_ans)):
    for word in clean_ans[line].split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1



vocab = {}
word_num = 0
for word, count in word2count.items():
    if count >= 4:
        vocab[word] = word_num
        word_num += 1
        
      



for i in range(len(clean_ans)):
    clean_ans[i] = '<SOS> ' + clean_ans[i] + ' <EOS>'



tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
x = len(vocab)
for token in range(len(tokens)):
    vocab[tokens[token]] = x
    x += 1
    
    

vocab['cameron'] = vocab['<PAD>']
vocab['<PAD>'] = 0


inv_vocab = {w:v for v, w in vocab.items()}





encoder_inp = []
for line in range(len(clean_ques)):
    lst = []
    for word in clean_ques[line].split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
        
    encoder_inp.append(lst)

decoder_inp = []
for line in range(len(clean_ans)):
    lst = []
    for word in clean_ans[line].split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])        
    decoder_inp.append(lst)






from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_inp = pad_sequences(encoder_inp, 15, padding='post', truncating='post')
decoder_inp = pad_sequences(decoder_inp, 15, padding='post', truncating='post')




decoder_final_output = []
for i in range(len(decoder_inp)):
    decoder_final_output.append(decoder_inp[i][1:]) 

decoder_final_output = pad_sequences(decoder_final_output, 15, padding='post', truncating='post')




from tensorflow.keras.utils import to_categorical
decoder_final_output = to_categorical(decoder_final_output, len(vocab))

In [ ]:
#Model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input


enc_inp = Input(shape=(15, ))
dec_inp = Input(shape=(15, ))


VOCAB_SIZE = len(vocab)
embed = Embedding(VOCAB_SIZE+1, output_dim=50, 
                  input_length=15,
                  trainable=True                  
                  )


enc_embed = embed(enc_inp)
enc_lstm = LSTM(400, return_sequences=True, return_state=True)
enc_op, h, c = enc_lstm(enc_embed)
enc_states = [h, c]



dec_embed = embed(dec_inp)
dec_lstm = LSTM(400, return_sequences=True, return_state=True)
dec_op, _, _ = dec_lstm(dec_embed, initial_state=enc_states)

dense = Dense(VOCAB_SIZE, activation='softmax')

dense_op = dense(dec_op)

model = Model([enc_inp, dec_inp], dense_op)




model.compile(loss='categorical_crossentropy',metrics=['acc'],optimizer='adam')

model.fit([encoder_inp, decoder_inp],decoder_final_output,epochs=5)

657/657 [==============================] - 225s 337ms/step - loss: 3.1429 - acc: 0.4992
Epoch 2/5
657/657 [==============================] - 221s 337ms/step - loss: 2.7883 - acc: 0.5298
Epoch 3/5
657/657 [==============================] - 220s 335ms/step - loss: 2.6323 - acc: 0.5480
Epoch 4/5
657/657 [==============================] - 221s 336ms/step - loss: 2.5396 - acc: 0.5547
Epoch 5/5
657/657 [==============================] - 222s 337ms/step - loss: 2.4755 - acc: 0.5581


In [ ]:
from tensorflow.keras.models import Model 
      

enc_model = Model([enc_inp], enc_states)



# decoder Model
decoder_state_input_h = Input(shape=(400,))
decoder_state_input_c = Input(shape=(400,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


decoder_outputs, state_h, state_c = dec_lstm(dec_embed , 
                                    initial_state=decoder_states_inputs)


decoder_states = [state_h, state_c]


dec_model = Model([dec_inp]+ decoder_states_inputs,
                                      [decoder_outputs]+ decoder_states)




from keras.preprocessing.sequence import pad_sequences
print("Conversation started")

import numpy as np
prepro1 = ""
while prepro1 != 'quit':
    prepro1  = input("Ask me : ")
   

    prepro1 = clean_text(prepro1)
   

    prepro = [prepro1]
   

    txt = []
    for x in prepro:
        # x = "hello"
        lst = []
        for y in x.split():
            ## y = "hello"
            try:
                lst.append(vocab[y])
                ## vocab['hello'] = 454
            except:
                lst.append(vocab['<OUT>'])
        txt.append(lst)

    ## txt = [[454]]
    txt = pad_sequences(txt, 15, padding='post')

    ## txt = [[454,0,0,0,.........13]]

    stat = enc_model.predict( txt)

    empty_target_seq = np.zeros( ( 1 , 1) )
     ##   empty_target_seq = [0]


    empty_target_seq[0, 0] = vocab['<SOS>']
    ##    empty_target_seq = [255]

    stop_condition = False
    decoded_translation = ''

    while not stop_condition :

        dec_outputs , h, c= dec_model.predict([ empty_target_seq] + stat )
        decoder_concat_input = dense(dec_outputs)
        ## decoder_concat_input = [0.1, 0.2, .4, .0, ...............]

        sampled_word_index = np.argmax( decoder_concat_input[0, -1, :] )
        ## sampled_word_index = [2]

        sampled_word = inv_vocab[sampled_word_index] + ' '

        ## inv_vocab[2] = 'hi'
        ## sampled_word = 'hi '

        if sampled_word != '<EOS> ':
            decoded_translation += sampled_word  

        if sampled_word == '<EOS> ' or len(decoded_translation.split()) > 15:
            stop_condition = True 

        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        ## <SOS> - > hi
        ## hi --> <EOS>
        stat = [h, c]  

    print("Chatbot initializing: ", decoded_translation )
    print("*****++++*****")

NameError: ignored